## [How to convert trained Keras model to a single TensorFlow .pb file and make prediction](https://www.dlology.com/blog/how-to-convert-trained-keras-model-to-tensorflow-and-make-prediction/)


- Train Keras model
- Keras to single TensorFlow .pb file
- Load .pb file with TensorFlow and make prediction.
- (Optional) Visualize the graph in Jupyter notebook.

## Create, compile and train model

In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 9s 149us/step - loss: 0.2691 - acc: 0.9170 - val_loss: 0.0540 - val_acc: 0.9830
Epoch 2/5
60000/60000 [==============================] - 6s 92us/step - loss: 0.0900 - acc: 0.9735 - val_loss: 0.0388 - val_acc: 0.9870
Epoch 3/5
60000/60000 [==============================] - 6s 92us/step - loss: 0.0670 - acc: 0.9803 - val_loss: 0.0354 - val_acc: 0.9881
Epoch 4/5
60000/60000 [==============================] - 5s 92us/step - loss: 0.0526 - acc: 0.9841 - val_loss: 0.0306 - val_acc: 0.9910
Epoch 5/5
60000/60000 [==============================] - 5s 91us/step - loss: 0.0469 - acc: 0.9860 - val_loss: 0.0328 - val_acc: 0.9891
Test loss: 0.03276554842220285
Test accuracy: 0.9891


### Save Keras model to a file (optional)

In [4]:
import os
os.makedirs('./model', exist_ok=True)
model.save('./model/keras_model.h5')

### Load Keras model from a file (optional)

In case you ran into the **incompatible with expected resource** issue with a model containing **BatchNormization** layers such as DenseNet, make sure to set the learning phase to 0 before loading the Keras model in a new session.
The error message looks like this when you call `tf.import_graph_def(graph_def)` later on during restoring. 
```
Conv_1/cond/ReadVariableOp/Switch was passed float from Conv_1/gamma:0 incompatible with expected resource
```

In [ ]:
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

In [1]:
from keras.models import load_model
model = load_model('./model/keras_model.h5')

Using TensorFlow backend.


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [7]:
from show_graph import show_graph
from keras import backend as K
import tensorflow as tf
sess = K.get_session()
graph_def = sess.graph.as_graph_def()
# graph_def
show_graph(graph_def)

### Check the input and output name
We will refer to them by name during TensorFlow inference.

In [9]:
print(model.outputs)

[<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]


In [10]:
print(model.inputs)

[<tf.Tensor 'conv2d_1_input:0' shape=(?, 28, 28, 1) dtype=float32>]


In [13]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [14]:
from keras import backend as K
import tensorflow as tf

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 27 variables.
INFO:tensorflow:Converted 27 variables to const ops.


Save `GraphDef` ProtoBuf as a binary file.

In [7]:
tf.train.write_graph(frozen_graph, "model", "tf_model.pb", as_text=False)

'model\\tf_model.pb'

## Load the pb file and inference
**Restart the Kernel first to clear previous training session.**

In [1]:
import tensorflow as tf
import os
import sys
from tensorflow.python.platform import gfile

In [2]:
sess=tf.InteractiveSession()

In [3]:
f = gfile.FastGFile("./model/tf_model.pb", 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
f.close()

In [4]:
sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
tf.import_graph_def(graph_def)

### Locate the output Tensor by name

In [5]:
softmax_tensor = sess.graph.get_tensor_by_name('import/dense_2/Softmax:0')

### Prepare datasets for inference

In [6]:
from keras.datasets import mnist
from keras import backend as K


# input image dimensions
img_rows, img_cols = 28, 28

(_, _), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_test = x_test.astype('float32')
x_test /= 255
print('x_test shape:', x_test.shape)
print(x_test.shape[0], 'test samples')

Using TensorFlow backend.


x_test shape: (10000, 28, 28, 1)
10000 test samples


### Make prediction

In [7]:
predictions = sess.run(softmax_tensor, {'import/conv2d_1_input:0': x_test[:20]})

In [8]:
[p.argmax() for p in predictions]

[7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4]

In [9]:
y_test[:20]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4],
      dtype=uint8)

### Loops through `NodeDef`s in `graph_def`
Members of a `NodeDef`:
* name
* op
* input
* device
* attr

In [10]:
for i, item in enumerate(graph_def.node):
    print(i, item.op, item.name, item.input)

0 Placeholder conv2d_1_input []
1 Const conv2d_1/kernel []
2 Identity conv2d_1/kernel/read ['conv2d_1/kernel']
3 Const conv2d_1/bias []
4 Identity conv2d_1/bias/read ['conv2d_1/bias']
5 Conv2D conv2d_1/convolution ['conv2d_1_input', 'conv2d_1/kernel/read']
6 BiasAdd conv2d_1/BiasAdd ['conv2d_1/convolution', 'conv2d_1/bias/read']
7 Relu conv2d_1/Relu ['conv2d_1/BiasAdd']
8 Const conv2d_2/kernel []
9 Identity conv2d_2/kernel/read ['conv2d_2/kernel']
10 Const conv2d_2/bias []
11 Identity conv2d_2/bias/read ['conv2d_2/bias']
12 Conv2D conv2d_2/convolution ['conv2d_1/Relu', 'conv2d_2/kernel/read']
13 BiasAdd conv2d_2/BiasAdd ['conv2d_2/convolution', 'conv2d_2/bias/read']
14 Relu conv2d_2/Relu ['conv2d_2/BiasAdd']
15 MaxPool max_pooling2d_1/MaxPool ['conv2d_2/Relu']
16 Const dropout_1/keras_learning_phase/input []
17 PlaceholderWithDefault dropout_1/keras_learning_phase ['dropout_1/keras_learning_phase/input']
18 Switch dropout_1/cond/Switch ['dropout_1/keras_learning_phase', 'dropout_1/kera

### List of operations in the graph

In [6]:
for op in sess.graph.get_operations():
    print(op.name, op.type)

import/conv2d_1_input Placeholder
import/conv2d_1/kernel Const
import/conv2d_1/kernel/read Identity
import/conv2d_1/bias Const
import/conv2d_1/bias/read Identity
import/conv2d_1/convolution Conv2D
import/conv2d_1/BiasAdd BiasAdd
import/conv2d_1/Relu Relu
import/conv2d_2/kernel Const
import/conv2d_2/kernel/read Identity
import/conv2d_2/bias Const
import/conv2d_2/bias/read Identity
import/conv2d_2/convolution Conv2D
import/conv2d_2/BiasAdd BiasAdd
import/conv2d_2/Relu Relu
import/max_pooling2d_1/MaxPool MaxPool
import/dropout_1/keras_learning_phase/input Const
import/dropout_1/keras_learning_phase PlaceholderWithDefault
import/dropout_1/cond/Switch Switch
import/dropout_1/cond/switch_t Identity
import/dropout_1/cond/pred_id Identity
import/dropout_1/cond/mul/y Const
import/dropout_1/cond/mul/Switch Switch
import/dropout_1/cond/mul Mul
import/dropout_1/cond/dropout/keep_prob Const
import/dropout_1/cond/dropout/Shape Shape
import/dropout_1/cond/dropout/random_uniform/min Const
import/dropo

## Visualize the `graph_def` in TensorBoard

In [7]:
from show_graph import show_graph

In [23]:
show_graph(tf.get_default_graph().as_graph_def())

In [6]:
# graph_def
show_graph(graph_def)